In [1]:
#Modified to save the distributions instead of just taking means

import pandas as pd
import numpy as np
from scipy.stats import wasserstein_distance, ks_2samp, entropy
from scipy.spatial.distance import jensenshannon
from sklearn.metrics import pairwise_distances
import os
import glob

# ========== CONFIGURATION ==========

variables = [
    "heartrate",
    "activity",
    "step_count",
    "activity_duration"
]

# ========== HELPER FUNCTIONS ==========


def compute_jsd(p, q, bins=50):
    """Fast Jensen–Shannon divergence for continuous data."""
    p, q = np.asarray(p, float), np.asarray(q, float)
    hist_range = (min(p.min(), q.min()), max(p.max(), q.max()))
    # Shared bin edges for fair comparison
    edges = np.linspace(hist_range[0], hist_range[1], bins + 1)
    p_hist, _ = np.histogram(p, bins=edges, density=True)
    q_hist, _ = np.histogram(q, bins=edges, density=True)
    p_hist /= p_hist.sum() + 1e-12
    q_hist /= q_hist.sum() + 1e-12
    m = 0.5 * (p_hist + q_hist)
    jsd = 0.5 * (entropy(p_hist, m) + entropy(q_hist, m))
    return np.sqrt(jsd)

def distance_correlation(x, y):
    """Approximate distance correlation using rank-based estimator (linear time)."""
    x, y = np.asarray(x, float), np.asarray(y, float)
    n = min(len(x), len(y))
    if n < 2:
        return np.nan
    x, y = x[:n], y[:n]
    rx = np.argsort(np.argsort(x))
    ry = np.argsort(np.argsort(y))
    cov_xy = np.cov(rx, ry, bias=True)[0, 1]
    stdx = np.std(rx)
    stdy = np.std(ry)
    return 0.0 if stdx * stdy == 0 else cov_xy / (stdx * stdy)
# ========== MAIN COMPARISON FUNCTION ==========
#
# def compare_data(df1, df2, label="comparison", match_datetimes=False, input_type='real'):
#     """Compare two DataFrames and return metric dictionary keyed by metric name."""
#     if any(v not in df1.columns or v not in df2.columns for v in variables):
#         raise ValueError("Both DataFrames must contain all variables of interest.")
#
#     # Handle datetimes
#     if 'Datetime' not in df1.columns or 'Datetime' not in df2.columns:
#         raise ValueError("Both CSVs must contain a 'Datetime' column.")
#     fmt1 = '%Y-%m-%d %H:%M:%S'
#     df1['Datetime'] = pd.to_datetime(df1['Datetime'], errors='coerce', format=fmt1)
#     fmt2= '%Y-%m-%d %H:%M:%S'
#     df2['Datetime'] = pd.to_datetime(df2['Datetime'], errors='coerce', format=fmt2)
#     df1, df2 = df1.dropna(subset=['Datetime']), df2.dropna(subset=['Datetime'])
#     df1 = df1.drop_duplicates(subset=['Datetime'], keep='first')
#     df2 = df2.drop_duplicates(subset=['Datetime'], keep='first')
#     # if match_datetimes:
#     #     df1 = df1[df1['Datetime'].isin(df2['Datetime'])].reset_index(drop=True)
#
#     # Initialize results per metric
#     metric_results = {
#         "Wasserstein": {"Comparison": label},
#         "KS_statistic": {"Comparison": label},
#         "JSD": {"Comparison": label},
#         "Distance_correlation": {"Comparison": label},
#     }
#
#     # Compute per-variable metrics
#     for var in variables:
#         # print(f"Comparing {var}...")
#         x, y = df1[var].dropna(), df2[var].dropna()
#         # print(x.shape, y.shape)
#         if len(x) == 0 or len(y) == 0:
#             for m in metric_results:
#                 metric_results[m][var] = np.nan
#             continue
#
#         if var == "Activity_Type" or x.dtype == object:
#             # Categorical → JSD only
#             p = x.value_counts(normalize=True)
#             q = y.value_counts(normalize=True)
#             common_idx = p.index.union(q.index)
#             p = p.reindex(common_idx, fill_value=0)
#             q = q.reindex(common_idx, fill_value=0)
#             jsd = jensenshannon(p, q)
#             metric_results["JSD"][var] = jsd
#             for m in ["Wasserstein", "KS_statistic", "Distance_correlation"]:
#                 metric_results[m][var] = np.nan
#         else:
#             # Continuous
#             metric_results["Wasserstein"][var] = wasserstein_distance(x, y)
#             metric_results["KS_statistic"][var] = ks_2samp(x, y).statistic
#             metric_results["JSD"][var] = compute_jsd(x, y)
#             metric_results["Distance_correlation"][var] = distance_correlation(x, y)
#
#     return metric_results
def compare_data_fast(df1, df2, variables, label="comparison"):
    """
    Compare two DataFrames of wearable variables using multiple distance metrics.
    Much faster than the original implementation.
    """
    # Initialize results
    metrics = ["Wasserstein", "KS_statistic", "JSD", "Distance_correlation"]
    metric_results = {m: {"Comparison": label} for m in metrics}

    # Ensure valid datetimes and remove duplicates

    for var in variables:
        if var not in df1.columns or var not in df2.columns:
            for m in metrics: metric_results[m][var] = np.nan
            continue

        x, y = df1[var].dropna(), df2[var].dropna()
        if len(x) == 0 or len(y) == 0:
            for m in metrics: metric_results[m][var] = np.nan
            continue

        # Handle categorical variables separately
        if var == "activity" or x.dtype == object:
            p = x.value_counts(normalize=True)
            q = y.value_counts(normalize=True)
            common = p.index.union(q.index)
            p = p.reindex(common, fill_value=0)
            q = q.reindex(common, fill_value=0)
            metric_results["JSD"][var] = jensenshannon(p, q)
            metric_results["Wasserstein"][var] = np.nan
            metric_results["KS_statistic"][var] = np.nan
            metric_results["Distance_correlation"][var] = np.nan
        else:
            xv, yv = x.values, y.values
            metric_results["Wasserstein"][var] = wasserstein_distance(xv, yv)
            metric_results["KS_statistic"][var] = ks_2samp(xv, yv).statistic
            metric_results["JSD"][var] = compute_jsd(xv, yv)
            metric_results["Distance_correlation"][var] = distance_correlation(xv, yv)

    return metric_results

# ========== CSV SAVING UTILITY ==========

def save_metric_csvs(all_metric_results, output_dir="metric_results"):
    """
    Combine all comparison results and save one CSV per metric.
    all_metric_results: list of dicts returned by compare_data()
    """
    os.makedirs(output_dir, exist_ok=True)

    metrics = ["Wasserstein", "KS_statistic", "JSD", "Distance_correlation"]

    for metric in metrics:
        df_metric = pd.DataFrame([r[metric] for r in all_metric_results])
        df_metric.to_csv(os.path.join(output_dir, f"{metric}_results.csv"), index=False)
        print(f"Saved {metric}_results.csv with {len(df_metric)} comparisons.")


# ========== EXAMPLE USAGE ==========
# for warmup in warmup_grid:
#     for alpha in alpha_grid:
#         print(f"alpha: {alpha}, warmup: {warmup}")
# synth_dir = f"sim_alpha_{alpha}_warmup_{warmup}"
synth_dir = r'C:\Users\Darren\Documents\BHI and Written Qualifier\tvae\samples_zufferey'
print(f"synth_dir: {synth_dir}")
output_dir = f"results_final_rev_tvae_zufferey"
os.makedirs(output_dir, exist_ok=True)
i=0
for file1 in glob.glob(os.path.join(synth_dir, '*.csv')):
    j=0
    synth_df_1 = pd.read_csv(file1)
    for file2 in glob.glob(os.path.join(synth_dir, '*.csv')):
        if file1 == file2:
            continue
        synth_df_2 = pd.read_csv(file2)
        # synth_df_1['Activity_Type'] = synth_df_1['Activity_Type'].map(activity_state_dict_inverse)
        # synth_df_2['Activity_Type'] = synth_df_2['Activity_Type'].map(activity_state_dict_inverse)
        results_df = pd.DataFrame(compare_data_fast(synth_df_1, synth_df_2, variables, label=f"{i}_vs_{i}"))
        results_df.to_csv(os.path.join(output_dir, f"{i}_vs_{j}.csv"))
        j+=1
    i+=1
#
#
#
#
# os.makedirs(output_dir, exist_ok=True)
# synth_dir = r'C:\Users\Darren\Documents\BHI and Written Qualifier\sim_alpha_0_warmup_0'
# self_comparison_result_list = []
# activity_state_dict_inverse = {val:key for key, val in activity_state_dict.items()}
# for i in range(len(data)):
#     # for j in range(i+1, len(data)):
#     #     print(f"Comparing {i} to {j}")
#     #     df = data[i].copy()
#     #     ref_df = data[j].copy()
#     #     df['Activity_Type'] = df['Activity_Type'].map(activity_state_dict_inverse)
#     #     ref_df['Activity_Type'] = ref_df['Activity_Type'].map(activity_state_dict_inverse)
#     #     results_df = pd.DataFrame(compare_data(df, ref_df, label=f"{i}_vs_{j}", input_type='real'))
#     #     results_df.to_csv(os.path.join(output_dir, f"{i}_vs_{j}.csv"))
#     #     self_comparison_result_list.append(results_df)
#     j = 0
#     for file in glob.glob(os.path.join(synth_dir, '*.csv')):
#         print(f'Comparing {i} to {j}')
#         ref_df = data[i].copy()
#         synth_df = pd.read_csv(file)
#         synth_df['Activity_Type'] = synth_df['Activity_Type'].map(activity_state_dict_inverse)
#         ref_df['Activity_Type'] = ref_df['Activity_Type'].map(activity_state_dict_inverse)
#         results_df = pd.DataFrame(compare_data(synth_df, ref_df, label=f"{i}_vs_{j}", match_datetimes = True, input_type='synth'))
#         # print(results_df)
#         results_df.to_csv(os.path.join(output_dir, f"{i}_vs_{j}.csv"))
#         j+=1
#


synth_dir: C:\Users\Darren\Documents\BHI and Written Qualifier\tvae\samples_zufferey


In [4]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# ========= CONFIGURATION =========
reference_dir = r'C:\Users\Darren\Documents\BHI and Written Qualifier\reference_results'
evaluation_dir = r'C:\Users\Darren\Documents\BHI and Written Qualifier\results_final_rev_tvae'
n_boot = 1000
output_csv = 'tvae_bootstrap_summary_bhi_final.csv'

# ========= HELPER FUNCTIONS =========

def load_metric_csv(path):
    """Load a metric CSV and drop non-numeric rows."""
    df = pd.read_csv(path, index_col=0)
    df = df.dropna(how='all')  # remove empty rows
    df = df.apply(pd.to_numeric, errors='coerce')
    return df

def summarize_bootstrap(diffs):
    """Compute summary statistics from a 3D array (bootstraps × vars × metrics)."""
    summary = {}
    for metric in diffs.columns:
        data = diffs[metric].dropna()
        summary[metric] = {
            'mean': data.mean(),
            'std': data.std(),
            'median': data.median(),
            'IQR': data.quantile(0.75) - data.quantile(0.25),
            'range': data.max() - data.min(),
            '95%_CI_low': np.percentile(data, 2.5),
            '95%_CI_high': np.percentile(data, 97.5)
        }
    return pd.DataFrame(summary).T

# ========= LOAD FILES =========

ref_files = sorted([os.path.join(reference_dir, f) for f in os.listdir(reference_dir) if f.endswith('.csv')])
eval_files = sorted([os.path.join(evaluation_dir, f) for f in os.listdir(evaluation_dir) if f.endswith('.csv')])

if len(ref_files) == 0 or len(eval_files) == 0:
    raise ValueError("No CSV files found in one or both folders.")

# ========= BOOTSTRAP =========

abs_boot = []  # list of DataFrames (abs differences)
rel_boot = []  # list of DataFrames (rel differences)

for _ in tqdm(range(n_boot), desc="Bootstrapping"):
    ref_path = np.random.choice(ref_files)
    eval_path = np.random.choice(eval_files)

    ref_df = load_metric_csv(ref_path)
    eval_df = load_metric_csv(eval_path)

    # Ensure same indices and columns
    common_rows = ref_df.index.intersection(eval_df.index)
    common_cols = ref_df.columns.intersection(eval_df.columns)
    ref_df = ref_df.loc[common_rows, common_cols]
    eval_df = eval_df.loc[common_rows, common_cols]

    abs_diff = (eval_df - ref_df).abs()
    rel_diff = abs_diff / ref_df.replace(0, np.nan).abs()  # avoid divide-by-zero

    abs_boot.append(abs_diff)
    rel_boot.append(rel_diff)

print(abs_boot[0])
print(rel_boot[0])
# ========= AGGREGATE RESULTS =========

abs_concat = pd.concat(abs_boot, axis=0, ignore_index=True)
rel_concat = pd.concat(rel_boot, axis=0, ignore_index=True)
abs_summary = summarize_bootstrap(abs_concat)
rel_summary = summarize_bootstrap(rel_concat)

# Combine and label
abs_summary['Type'] = 'Absolute'
rel_summary['Type'] = 'Relative'
summary = pd.concat([abs_summary, rel_summary])

# ========= SAVE =========
summary.to_csv(output_csv, index_label='Metric')
print(f"Bootstrap summary saved to {output_csv}")

# ========= OPTIONAL: Display =========
print(summary)


Bootstrapping: 100%|██████████| 1000/1000 [00:04<00:00, 218.12it/s]

                             Wasserstein  KS_statistic       JSD  \
Comparison                           NaN           NaN       NaN   
Activity_Type                        NaN           NaN  0.149942   
Heart rate___beats/minute       9.764735      0.165744  0.281103   
Calories burned_kcal           11.016143      0.146368  0.328946   
Exercise duration_s           116.559992      0.170834  0.383361   
Sleep duration_minutes          4.605763      0.180928  0.261634   
Sleep type duration_minutes     2.943984      0.215605  0.280553   
Floors climbed___floors         0.000000      0.000000       NaN   

                             Distance_correlation  
Comparison                                    NaN  
Activity_Type                                 NaN  
Heart rate___beats/minute                0.034765  
Calories burned_kcal                     0.076991  
Exercise duration_s                      0.164178  
Sleep duration_minutes                   0.111097  
Sleep type duration_min

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import jensenshannon

# ========= CONFIGURATION =========
reference_dir = (r'C:\Users\Darren\Documents\BHI and Written Qualifier\ctgan\reference\comparison_summary_zufferey')
evaluation_dir = r'C:\Users\Darren\Documents\BHI and Written Qualifier\ctgan\results_final_rev_ctgan_zufferey'
n_boot = 1000
n_sample = 1000  # number of CSVs sampled per bootstrap
output_csv = 'ctgan_bootstrap_WD_summary_fitbit.csv'

metric_files = [
    "Wasserstein_results.csv",
    "JSD_results.csv",
    "KS_results.csv",
    "DistCorr_results.csv"
]

# ========= HELPER FUNCTIONS =========
def load_metric_csv(path):
    """Load a metric CSV and drop non-numeric rows."""
    df = pd.read_csv(path, index_col=0)
    df = df.dropna(how='all')
    df = df.apply(pd.to_numeric, errors='coerce')
    return df

def summarize_bootstrap(df):
    """Compute summary stats for WD/JSD results."""
    summary = (
        df.groupby(['Variable', 'Metric'])
          .agg(['mean', 'std', 'median',
                lambda x: x.quantile(0.25),
                lambda x: x.quantile(0.75),
                np.min,
                np.max,  # range
                lambda x: np.percentile(x, 2.5),  # 95% CI low
                lambda x: np.percentile(x, 97.5)])  # 95% CI high
    )
    summary.columns = ['mean', 'std', 'median','IQR_25', 'IQR_75', 'min', 'max', '95%_CI_low', '95%_CI_high']
    return summary.reset_index()

def compute_jsd(p, q, bins=50):
    """Compute Jensen–Shannon distance between continuous samples."""
    if len(p) == 0 or len(q) == 0:
        return np.nan
    min_v = min(np.min(p), np.min(q))
    max_v = max(np.max(p), np.max(q))
    if not np.isfinite(min_v) or not np.isfinite(max_v) or min_v == max_v:
        return np.nan
    hist_p, _ = np.histogram(p, bins=bins, range=(min_v, max_v), density=True)
    hist_q, _ = np.histogram(q, bins=bins, range=(min_v, max_v), density=True)
    hist_p += 1e-12
    hist_q += 1e-12
    hist_p /= hist_p.sum()
    hist_q /= hist_q.sum()
    return jensenshannon(hist_p, hist_q)

# ========= LOAD REFERENCE METRIC FILES =========
ref_metrics = {}
for mfile in metric_files:
    metric_name = mfile.split("_")[0]  # e.g., "Wasserstein" from "Wasserstein_results.csv"
    path = os.path.join(reference_dir, mfile)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Reference file not found: {path}")
    if metric_name == 'DistCorr':
        metric_name = 'Distance_correlation'
    if metric_name == 'KS':
        metric_name = 'KS_statistic'
    ref_metrics[metric_name] = load_metric_csv(path)

print(f"Loaded reference metrics: {list(ref_metrics.keys())}")

# ========= LOAD EVALUATION FILES =========
eval_files = [
    load_metric_csv(os.path.join(evaluation_dir, f))
    for f in os.listdir(evaluation_dir)
    if f.endswith('.csv') and '_vs_' in f
]

if len(eval_files) == 0:
    raise ValueError("No evaluation {i}_vs_{j}.csv files found in the directory.")

# ========= DETERMINE COMMON VARIABLES =========
common_vars = list(set.intersection(*[set(df.columns) for df in ref_metrics.values()]))
print(f"Found {len(common_vars)} variables shared across all reference metrics.")

# ========= BOOTSTRAP COMPARISON =========
records = []

for i in tqdm(range(n_boot), desc="Bootstrapping WD/JSD"):
    eval_sample_idxs = np.random.choice(len(eval_files), size=n_sample, replace=True)

    for metric_name, ref_df in ref_metrics.items():
        ref_vals_all = ref_df[common_vars]
        ref_vals_all = ref_vals_all.select_dtypes(include=[np.number])

        for var in common_vars:
            ref_vals = ref_vals_all[var].dropna().values
            if len(ref_vals) < 2:
                continue

            # Gather corresponding metric column from sampled eval files
            eval_vals = []
            for idx in eval_sample_idxs:
                try:
                    eval_df = eval_files[idx]
                    if metric_name in eval_df.columns and var in eval_df.index:
                        val = eval_df.at[var, metric_name]
                        if np.isfinite(val):
                            eval_vals.append(val)
                except Exception:
                    continue

            if len(eval_vals) < 2:
                continue

            eval_vals = np.array(eval_vals, dtype=float)
            ref_vals = np.array(ref_vals, dtype=float)

            # Compute WD and JSD between the bootstrap-sampled distributions
            wd = wasserstein_distance(ref_vals, eval_vals)
            jsd = compute_jsd(ref_vals, eval_vals)

            records.append({
                "Variable": var,
                "Metric": metric_name,
                "Wasserstein": wd
                # "JSD": jsd
            })

# ========= SUMMARIZE =========
df_results = pd.DataFrame(records)

summary_wd = summarize_bootstrap(
    df_results[['Variable', 'Metric', 'Wasserstein']].rename(columns={'Wasserstein': 'Value'})
)
# summary_jsd = summarize_bootstrap(
#     df_results[['Variable', 'Metric', 'JSD']].rename(columns={'JSD': 'Value'})
# )

summary_wd['Distance'] = 'Wasserstein'
# summary_jsd['Distance'] = 'JSD'

summary = summary_wd
# summary = pd.concat([summary_wd, summary_jsd], ignore_index=True)
summary = summary[['Variable', 'Metric', 'Distance','mean', 'std', 'median','IQR_25', 'IQR_75', 'min', 'max', '95%_CI_low', '95%_CI_high']]


# ========= SAVE =========
summary.to_csv(output_csv, index=False)
print(f"\n✅ Bootstrap WD/JSD summary saved to {output_csv}")
print(summary.head(15))

Loaded reference metrics: ['Wasserstein', 'JSD', 'KS_statistic', 'Distance_correlation']
Found 4 variables shared across all reference metrics.


Bootstrapping WD/JSD: 100%|██████████| 1000/1000 [01:18<00:00, 12.76it/s]


✅ Bootstrap WD/JSD summary saved to ctgan_bootstrap_WD_summary_fitbit.csv
             Variable                Metric     Distance       mean       std  \
0            activity                   JSD  Wasserstein   0.505340  0.000080   
1   activity_duration  Distance_correlation  Wasserstein   0.182815  0.000224   
2   activity_duration                   JSD  Wasserstein   0.471111  0.000079   
3   activity_duration          KS_statistic  Wasserstein   0.263113  0.000077   
4   activity_duration           Wasserstein  Wasserstein  14.575900  0.004048   
5           heartrate  Distance_correlation  Wasserstein   0.230525  0.000230   
6           heartrate                   JSD  Wasserstein   0.472832  0.000080   
7           heartrate          KS_statistic  Wasserstein   0.325289  0.000072   
8           heartrate           Wasserstein  Wasserstein   8.683148  0.002313   
9          step_count  Distance_correlation  Wasserstein   0.214065  0.000232   
10         step_count             


C:\Users\Darren\AppData\Local\Temp\ipykernel_3148\2131998029.py:34: FutureWarning: The provided callable <function min at 0x000001BF78D72C20> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg(['mean', 'std', 'median',
C:\Users\Darren\AppData\Local\Temp\ipykernel_3148\2131998029.py:34: FutureWarning: The provided callable <function max at 0x000001BF78D72B00> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  .agg(['mean', 'std', 'median',


In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import jensenshannon

# ========= CONFIGURATION =========
reference_dir = r'C:\Users\Darren\Documents\BHI and Written Qualifier\reference_results'
evaluation_dir = r'C:\Users\Darren\Documents\BHI and Written Qualifier\results_final_rev_tvae'

n_boot = 1000
n_sample = 1000
output_csv = 'tvae_bootstrap_WD_summary_final_bhi.csv'

METRICS = ["Wasserstein", "KS_statistic", "JSD", "Distance_correlation"]


# ========= HELPER FUNCTIONS =========
def load_vs_csv(path):
    """Load file with index=variables and metric columns."""
    df = pd.read_csv(path, index_col=0)
    df = df.apply(pd.to_numeric, errors='coerce')
    return df


def compute_jsd(p, q, bins=50):
    if len(p) == 0 or len(q) == 0:
        return np.nan
    low = min(np.min(p), np.min(q))
    high = max(np.max(p), np.max(q))
    if not np.isfinite(low) or not np.isfinite(high) or low == high:
        return np.nan

    hist_p, _ = np.histogram(p, bins=bins, range=(low, high), density=True)
    hist_q, _ = np.histogram(q, bins=bins, range=(low, high), density=True)
    hist_p = np.maximum(hist_p, 1e-12)
    hist_q = np.maximum(hist_q, 1e-12)
    hist_p /= hist_p.sum()
    hist_q /= hist_q.sum()
    return jensenshannon(hist_p, hist_q)


def summarize_bootstrap(df):
    summary = (
        df.groupby(['Variable', 'Metric'])
          .agg(['mean', 'std', 'median',
                lambda x: x.quantile(0.25),
                lambda x: x.quantile(0.75),
                np.min,
                np.max,
                lambda x: np.percentile(x, 2.5),
                lambda x: np.percentile(x, 97.5)])
    )
    summary.columns = ['mean', 'std', 'median', 'IQR_25', 'IQR_75',
                       'min', 'max', '95%_CI_low', '95%_CI_high']
    return summary.reset_index()


# ========= LOAD REFERENCE & EVAL FILES =========
def load_directory_vs_files(folder):
    return [
        load_vs_csv(os.path.join(folder, f))
        for f in os.listdir(folder)
        if f.endswith('.csv') and '_vs_' in f
    ]

ref_files = load_directory_vs_files(reference_dir)
eval_files = load_directory_vs_files(evaluation_dir)

if len(ref_files) == 0:
    raise ValueError("No reference *_vs_*.csv files found.")
if len(eval_files) == 0:
    raise ValueError("No evaluation *_vs_*.csv files found.")

# ========= Determine common variables across all CSVs =========
common_vars = list(
    set.intersection(*[set(df.index) for df in ref_files])
)
print(f"Common variables: {common_vars}")

# ========= BOOTSTRAP =========
records = []

for b in tqdm(range(n_boot), desc="Bootstrapping"):
    ref_idx = np.random.choice(len(ref_files), size=n_sample, replace=True)
    eval_idx = np.random.choice(len(eval_files), size=n_sample, replace=True)

    for metric_name in METRICS:
        for var in common_vars:

            # Collect bootstrap samples
            ref_vals = []
            for i in ref_idx:
                try:
                    v = ref_files[i].at[var, metric_name]
                    if np.isfinite(v): ref_vals.append(v)
                except Exception:
                    pass

            eval_vals = []
            for j in eval_idx:
                try:
                    v = eval_files[j].at[var, metric_name]
                    if np.isfinite(v): eval_vals.append(v)
                except Exception:
                    pass

            if len(ref_vals) < 2 or len(eval_vals) < 2:
                continue

            ref_vals = np.array(ref_vals, float)
            eval_vals = np.array(eval_vals, float)

            # Compute Wasserstein only (JSD optional)
            wd = wasserstein_distance(ref_vals, eval_vals)

            records.append({
                "Variable": var,
                "Metric": metric_name,
                "Distance": "Wasserstein",
                "Value": wd
            })

# ========= SUMMARY OUTPUT =========
df_results = pd.DataFrame(records)

summary = summarize_bootstrap(
    df_results[['Variable', 'Metric', 'Value']])
summary['Distance'] = 'Wasserstein'
summary = summary[['Variable', 'Metric', 'Distance', 'mean', 'std', 'median',
                   'IQR_25', 'IQR_75', 'min', 'max', '95%_CI_low', '95%_CI_high']]

summary.to_csv(output_csv, index=False)
print(f"\n✅ Saved bootstrap summary → {output_csv}")
print(summary.head(10))


Common variables: ['Floors climbed___floors', 'Heart rate___beats/minute', 'Sleep type duration_minutes', 'Activity_Type', 'Calories burned_kcal', 'Comparison', 'Exercise duration_s', 'Sleep duration_minutes']


Bootstrapping: 100%|██████████| 1000/1000 [04:21<00:00,  3.83it/s]


TypeError: agg function failed [how->mean,dtype->object]